# Motor Testing April 6th

On April 6th 2025, we took Brightside out for driving/testing for the first time since FSGP 2024.

Due to the rain, tests were conducted in the undercover parking lot nearest to EDC. While we spent most of the testing time debugging, we were able to conduct a small number of test drives.

See https://docs.google.com/document/d/1Rx3J4bNzHDbjxchzbmqbx4wmzVHT7njctfJhgrXQq_g/edit for complete details.

## Cruising Test

We conducted a very brief cruising test at 2:59PM in which we got Chris to floor the accelerator, then release it and finally brake when necessary (the parking lot is not very long).

This test was conducted under our [Cruising Test Plan](https://docs.google.com/document/d/1nRl-FxTlJjzzBGooMMIHlvJZYS2zg53nI7BmjzIfTFw/edit?tab=t.0#heading=h.x81rs6u2klim). See all STG's plans [here](https://drive.google.com/drive/u/1/folders/1l5ZTJP1LZwLBvmLuFI7MEXHikJNSyDR0).

Video: https://drive.google.com/file/d/1kIt3Dud8n91xUy1W2zfV0xKJm5OHyfYW/view?usp=drive_link


### 1. Query Data From Influx

Thanks to Aarjav for quickly uploading the Memorator data to InfluxDB on the elec bay computer.